```python
from google.colab import drive
drive.mount('/content/drive')
import os
DIR = "/content/drive/MyDrive/Colab Notebooks/MNA/TC4029 - Ciencia y analítica de datos/Semana 6/Actividad6_FE"
os.chdir(DIR)
```

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import json
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import time
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from sklearn.preprocessing import MinMaxScaler

# Steam user items

In [44]:
steam_users = pd.read_json('steam_users_p3.json', lines=True)

In [4]:
working_df = steam_users.explode('items')
user_items_df = pd.json_normalize(working_df['items'])
user_items_df = user_items_df.join(working_df[['user_id', 'user_url']].reset_index(drop=True))


# Steam australian reviews.

In [51]:
steam_aus_rev_df = pd.read_json('MNA_GVD24/steam_aus_rev_p3.json', lines=True)

In [48]:
working_df = steam_aus_rev_df.explode('reviews')
reviews_df = pd.json_normalize(working_df['reviews'])
reviews_df = reviews_df.join(working_df[['user_id', 'user_url']].reset_index(drop=True))

/var/folders/5n/lp3_lp252c17c644s2777gmr0000gn/T/ipykernel_44616/648291562.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games_df['item_id'] =  pd.to_numeric(steam_games_df['item_id'], downcast='integer')


# Steam games df 

In [53]:
steam_games_df = pd.read_json('steam_games_p3.json', lines=True)
steam_games_df = steam_games_df.rename(columns={'id': 'item_id'})

In [54]:
steam_games_df = steam_games_df.drop(columns=['metascore'], axis=1)
steam_games_df['publisher'].fillna('', inplace=True)
steam_games_df['genres'].fillna('[]', inplace=True)  
steam_games_df['tags'].fillna('[]', inplace=True) 

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,,"Posted November 5, 2011.",,1250.0,No ratings yet,1,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,,"Posted July 15, 2011.",,22200.0,No ratings yet,1,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,,"Posted April 21, 2011.",,43110.0,No ratings yet,1,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,,"Posted June 24, 2014.",,251610.0,15 of 20 people (75%) found this review helpful,1,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,,"Posted September 8, 2013.",,227300.0,0 of 1 people (0%) found this review helpful,1,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637
...,...,...,...,...,...,...,...,...,...
59300,,Posted July 10.,,70.0,No ratings yet,1,a must have classic from steam definitely wort...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
59301,,Posted July 8.,,362890.0,No ratings yet,1,this game is a perfect remake of the original ...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
59302,1 person found this review funny,Posted July 3.,,273110.0,1 of 2 people (50%) found this review helpful,1,had so much fun plaing this and collecting res...,LydiaMorley,http://steamcommunity.com/id/LydiaMorley
59303,,Posted July 20.,,730.0,No ratings yet,1,:D,LydiaMorley,http://steamcommunity.com/id/LydiaMorley


In [55]:
steam_games_df.to_json('steam_games_df.json', orient='records', lines=True)
reviews_df.to_json('reviews_df.json', orient='records', lines=True)
user_items_df.to_json('user_items_df.json', orient='records', lines=True)


,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,item_id,developer,sentiment
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,None
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,None
4,,[],Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32128,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.49,http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS""",None
32129,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",4.24,http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada,None
32130,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",1.39,http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich,None
32131,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",NaN,http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns",1 user reviews


## Preprocesamiento

In [33]:
# reviews
reviews_df = reviews_df.dropna(subset=['recommend'])
assert reviews_df['recommend'].isna().sum() == 0
reviews_df['recommend'] = reviews_df['recommend'].astype(int)

# steam games
steam_games_df = steam_games_df[steam_games_df['item_id'].notna()]
steam_games_df.reset_index(drop=True, inplace=True)

steam_games_df['item_id'] = steam_games_df['item_id'].astype(int)

reviews_df = reviews_df[reviews_df['item_id'].notna()]
reviews_df.reset_index(drop=True, inplace=True)
reviews_df['item_id'] = reviews_df['item_id'].astype(int)
reviews_df = reviews_df[reviews_df['item_id'].isin(steam_games_df['item_id'])]
reviews_df = reviews_df.drop(columns=['funny', 'posted', 'last_edited', 'helpful', 'user_url'], axis=1)

0        761140.0
1        643980.0
2        670290.0
3        767400.0
4        773570.0
           ...   
32130    773640.0
32131    733530.0
32132    610660.0
32133    658870.0
32134    681550.0
Name: item_id, Length: 32135, dtype: float64

/var/folders/5n/lp3_lp252c17c644s2777gmr0000gn/T/ipykernel_44616/2463571085.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_items_df['log_ptime_scaled'] = scaler_playtime.fit_transform(user_items_df[['log_ptime']].values.reshape(-1, 1))


KeyError: "None of [Index(['exp_ptime_last2week'], dtype='object')] are in the [columns]"

In [ ]:

# user items

# Hay ids en perfiles que no estan en nuestros datos mundiales de steam_games... La base de datos tiene este error.
user_items_df = user_items_df[user_items_df['item_id'].isin(steam_games_df['item_id'])]
user_items_df.reset_index(drop=True, inplace=True)

user_items_df['log_ptime'] = np.log1p(user_items_df['playtime_forever'])

user_items_df['exp_ptime_2week'] = np.exp(user_items_df['playtime_2weeks']) - 1

In [ ]:
steam_games_df['item_id']

In [ ]:

scaler_playtime = MinMaxScaler(feature_range=(1, 5))
user_items_df['log_ptime_scaled'] = scaler_playtime.fit_transform(user_items_df[['log_ptime']].values.reshape(-1, 1))

scaler_lastweek = MinMaxScaler(feature_range=(1, 5))
user_items_df['exp_ptime_2week_scaled'] = scaler_lastweek.fit_transform(user_items_df[['exp_ptime_last2week']].values.reshape(-1, 1))

# Rating de usuario

La idea aquí es muy simple. Estamos dando 'importancia' a dos aspectos diferentes del comportamiento del usuario:

1. **El tiempo total de juego (`log_ptime_scaled`)**, que indica el interés general y la dedicación a lo largo del tiempo.
2. **El tiempo de juego en las últimas dos semanas (`exp_ptime_last2week_scaled`)**, que nos da una medida de la actividad reciente y la relevancia actual del juego para el usuario.

Al combinar estos dos factores en un único 'rating', podemos crear una métrica que equilibra tanto la lealtad a largo plazo como la participación actual. Esto es particularmente útil para sistemas de recomendación, donde queremos ajustar nuestras recomendaciones no solo en base a lo que un usuario ha disfrutado históricamente, sino también basándonos en lo que es más relevante para ellos recientemente.

Aquí, 'n' y 'm' son pesos asignados a la puntuación de juego reciente y la puntuación de juego total respectivamente. Estos pesos pueden ajustarse para enfatizar más uno u otro aspecto según la dinámica del usuario y los objetivos del sistema de recomendación.

In [ ]:
n=0.7 # Peso para el tiempo de juego en las últimas dos semanas
m=0.3 # Peso para el tiempo de juego total
user_items_df['combined_rating'] = n * user_items_df['exp_ptime_last2week_scaled'] + m * user_items_df['log_ptime_scaled']


In [ ]:
user_game_matrix = user_items_df.pivot_table(index='user_id', columns='item_id', values='combined_rating', fill_value=0)


## Contador

In [ ]:
contador_generos = Counter(g for genres in steam_games_df['genres'] for g in genres)
contador_tags = Counter(t for tags in steam_games_df['tags'] for t in tags)

## Transformado de datos


In [ ]:
genres = list(contador_generos.keys())

for g in genres:
    steam_games_df[g] = steam_games_df['genres'].transform(lambda x: int(g in x))
steam_games_df[genres].head()

tags = list(contador_tags.keys())

tags_matrix = {tag: steam_games_df['tags'].apply(lambda x: int(tag in x)) for tag in contador_tags.keys()}
tags_df = pd.DataFrame(tags_matrix)
steam_features = pd.concat([steam_games_df[genres], tags_df], axis=1)

steam_features.head()

## Merge features

In [ ]:
# Merge steam_features with steam_games_df using the standardized 'item_id'
steam_features = steam_features.merge(steam_games_df[['item_id']], left_index=True, right_on='item_id')
steam_features.set_index('item_id', inplace=True)  # Set 'item_id' as the index after the merge

# Join the user-game matrix with the steam features using the standardized 'item_id'
full_features = user_game_matrix.join(steam_features, on='item_id')  # Use 'item_id' for joining

# Fill NaN values where user hasn't rated
full_features.fillna(0, inplace=True) 

## Algoritmo de recomendación Similaridad de Coseno


In [ ]:
start_time_cosine_sim = time.time()
cosine_sim = cosine_similarity(full_features, full_features)
end_time_cosine_sim = time.time()

item_idx = dict(zip(steam_games_df['item_id'], list(steam_games_df.index)))

### Funciones Recomendación

In [ ]:
def encuentra_juego(item_id):
    title = steam_games_df[steam_games_df['item_id'] == item_id]['title'].iloc[0]
    return title

def obtener_recomendaciones_basadas_contenido(item_id, numero_recomendaciones=10):
    title = encuentra_juego(item_id)
    idx = item_idx[item_id]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:(numero_recomendaciones + 1)]
    similar_games = [i[0] for i in sim_scores]
    print(f"Porque te gustó {title} te pueden gustar:")
    print(steam_games_df['title'].iloc[similar_games])

def get_recom_content_eval(item_id, recom=5):
    #title = encuentra_juego(item_id)
    idx = item_idx[item_id]
    sim_scores = cosine_sim[idx]
    recom_eval = pd.DataFrame({'index': steam_games_df.index, 'similarity': sim_scores})
    #similarity_df = pd.DataFrame(list(enumerate(cosine_sim[idx])), columns=['index', 'similarity'])

    recom_eval = recom_eval.nlargest(recom, 'similarity')
    recom_eval['item_id'] = recom_eval['index'].map(steam_games_df['item_id'])

    return recom_eval

def get_recom_content_user(item_id, numero_recom=5):
    full_recom_df = get_recom_content_eval(item_id, numero_recom + 1)
    if item_id in full_recom_df['item_id'].values:
        full_recom_df = full_recom_df[full_recom_df['item_id'] != item_id]

    return full_recom_df.head(numero_recom)


## Métricas

In [ ]:

def gen_recom_ground_truth(numero_recom):
    recommendations = {}
    ground_truth = {}

    for user_id in reviews_df['user_id'].unique():
        user_reviews = reviews_df[reviews_df['user_id'] == user_id]
        ground_truth[user_id] = user_reviews[user_reviews['recommend'] == 1]['item_id'].tolist()

        user_recommendations = set()
        for item_id in user_reviews['item_id']:
            recommendations_for_game = get_recom_content_eval(item_id, numero_recom)
            if not recommendations_for_game.empty:
                user_recommendations.update(recommendations_for_game['item_id'].tolist())

        recommendations[user_id] = list(user_recommendations)[:numero_recom]

    return recommendations, ground_truth

def calcular_rmse(recommendations, ground_truth):
    mse_sum = 0
    total_users = 0

    for user_id in recommendations:
        recommended_games = set(recommendations[user_id])
        actual_games = set(ground_truth[user_id])

        # Calculamos los errores
        errors = [1 if game not in actual_games else 0 for game in recommended_games]

        # Si no hay recomendaciones reales para el usuario, continuamos
        if len(errors) == 0:
            continue

        mse_sum += np.mean(errors)
        total_users += 1

    # Calculamos el RMSE
    rmse = np.sqrt(mse_sum / total_users)

    return rmse

def precision_at_k(actual, predicted, k=10):
    if not actual:
        return 0.0

    predicted = predicted[:k]
    relevant = len(set(predicted) & set(actual))
    return relevant / k


def recall_at_k(actual, predicted, k=10):
    if not actual:
        return 0.0

    predicted = predicted[:k]
    relevant = len(set(predicted) & set(actual))
    return relevant / len(actual)


def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)


def mapk(actual, predicted, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])


def ndcg_at_k(actual, predicted, k=10):
    if not actual:
        return 0.0

    predicted = predicted[:k]
    idcg = sum([1.0 / np.log2(i + 2) for i in range(len(actual))])
    dcg = sum([1.0 / np.log2(i + 2) for i, p in enumerate(predicted) if p in actual])
    return dcg / idcg


def mrr_at_k(actual, predicted, k=10):
    if not actual:
        return 0.0

    predicted = predicted[:k]
    for i, p in enumerate(predicted):
        if p in actual:
            return 1.0 / (i + 1.0)
    return 0.0


def calculate_rmse(sparse_matrix, k):
    U, sigma, VT = svds(sparse_matrix, k=k)
    sigma = np.diag(sigma)
    predicted_ratings = np.dot(np.dot(U, sigma), VT)

    actual_ratings = sparse_matrix.toarray().flatten()
    predicted_ratings = predicted_ratings.flatten()

    mask = actual_ratings > 0
    actual_ratings = actual_ratings[mask]
    predicted_ratings = predicted_ratings[mask]

    rmse = sqrt(mean_squared_error(actual_ratings, predicted_ratings))
    return rmse


def evaluate_recommendations(ground_truth, recommendations, k=5):
    metrics = {
        'Precision@K': np.mean([precision_at_k(a, p, k) for a, p in zip(ground_truth, recommendations)]),
        'Recall@K': np.mean([recall_at_k(a, p, k) for a, p in zip(ground_truth, recommendations)]),
        'MAP@K': mapk(ground_truth, recommendations, k),
        'NDCG@K': np.mean([ndcg_at_k(a, p, k) for a, p in zip(ground_truth, recommendations)]),
        'MRR@K': np.mean([mrr_at_k(a, p, k) for a, p in zip(ground_truth, recommendations)])
    }
    return metrics


In [ ]:
start_time_rec_cosine = time.time()
recommendations_dict, ground_truth_dict = gen_recom_ground_truth(numero_recom=5)
end_time_rec_cosine = time.time()

# Cosine Similarity
recommendations = [recommendations_dict[user] for user in recommendations_dict]
ground_truth = [ground_truth_dict[user] for user in ground_truth_dict]

rmse_cosine = calcular_rmse(recommendations_dict, ground_truth_dict)
print(f"RMSE: {rmse_cosine}")

metrics_cosine = evaluate_recommendations(ground_truth, recommendations, k=5)
duration_training_cosine = end_time_cosine_sim - start_time_cosine_sim
duration_recommendation_cosine = end_time_rec_cosine - start_time_rec_cosine

metrics_df_simple = pd.DataFrame({
    'Metric': ['Precision@K', 'Recall@K', 'MAP@K', 'NDCG@K', 'MRR@K', 'RMSE',
               'Tiempo Entrenamiento', 'Tiempo Recomendación'],
    'Cosine Method': [
        metrics_cosine['Precision@K'],
        metrics_cosine['Recall@K'],
        metrics_cosine['MAP@K'],
        metrics_cosine['NDCG@K'],
        metrics_cosine['MRR@K'],
        rmse_cosine,
        duration_training_cosine,
        duration_recommendation_cosine
     ]
    # ,'SVD Method': [
    #     metrics_svd['Precision@K'],
    #     metrics_svd['Recall@K'],
    #     metrics_svd['MAP@K'],
    #     metrics_svd['NDCG@K'],
    #     metrics_svd['MRR@K'],
    #     rmse_svd,
    #     duration_training_svd,
    #     duration_recommendation_svd
    # ]
})
metrics_df_simple